# Notkun gervigreindar fyrir greiningu á þrívíddarmyndum

Nathan Holmes-King

In [40]:
import numpy as np
import pandas as pd
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")#"mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að greina þrívíddarmyndir. Í notkun eru líkön sem geta það, en þau nota alltaf "bitmap"-myndir. Þetta líkan hér notar "vector"-myndir eins og Envalys.

## Gögn
Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [41]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [42]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open('/Users/002-nathan/Desktop/Envalys/STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename='/Users/002-nathan/Desktop/Envalys/STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við deilum myndinni í 2.097.152 (128x128x128) þrívíddardíla eða "voxels", teljum punktana í hverjum díl, og notum töluna til að greina myndina.

In [55]:
X_data = []
y_data = []
for cat in skrar:
    print(cat)
    byrjun = time.time()
    for fi in skrar[cat]:
        # Load data
        gogn = mesh.Mesh.from_file('/Users/002-nathan/Desktop/Envalys/STLdata/' + cat + '_' + fi)
        # Re-scale to be 
        gogn_x = [a[0] for a in gogn.v0]# + [a[0] for a in gogn.v1] + [a[0] for a in gogn.v2]
        gogn_y = [a[1] for a in gogn.v0]# + [a[1] for a in gogn.v1] + [a[1] for a in gogn.v2]
        gogn_z = [a[2] for a in gogn.v0]# + [a[2] for a in gogn.v1] + [a[2] for a in gogn.v2]
        minx = min(gogn_x)
        miny = min(gogn_y)
        minz = min(gogn_z)
        scale = min(128 / (max(gogn_x) + 1e-3 - minx), 
                    128 / (max(gogn_y) + 1e-3 - miny), 
                    128 / (max(gogn_z) + 1e-3 - minz))
        ny_gogn = np.zeros((1, 128, 128, 128), dtype=np.float32)
        for a in gogn.v0:
            x = int((a[0] - minx) * scale)
            y = int((a[1] - miny) * scale)
            z = int((a[2] - minz) * scale)
            ny_gogn[0][x][y][z] += 1
        X_data.append(torch.from_numpy(ny_gogn).to(device))
        y_data.append(torch.tensor([catnum[cat]]).to(device))
    print(time.time() - byrjun)
    print('----')

body parts
44.615397930145264
----
buildings
26.1183819770813
----
geometric shapes
33.361220836639404
----
objects in space
exception (False, 'No lines found, impossible to read')
46.414735317230225
----
sculptures
116.57011818885803
----


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data)

## Líkan

In [57]:
class likan(nn.Module):
    def __init__(self):
        super(likan, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, 5, padding=2)
        self.conv2 = nn.Conv3d(16, 32, 5, padding=2)
        self.conv3 = nn.Conv3d(32, 64, 5, padding=2)
        self.fc1 = nn.Linear(64*16*16*16, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 5)  # Change second param
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool3d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool3d(x, 2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool3d(x, 2)
        x = x.view(-1, 64*16*16*16)
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [58]:
num_epochs = 10
learning_rate = 0.01
momentum = 0.5

In [59]:
model = likan()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [60]:
byrjun = time.time()
for i in range(num_epochs):
    print('Epoch: ', i)
    for n in range(len(X_train)):
        optimizer.zero_grad()
        output = model(X_train[n])
        loss = F.nll_loss(output, y_train[n])
        loss.backward()
        optimizer.step()
    correct = 0
    with torch.no_grad():
        for n in range(len(X_test)):
            output = model(X_test[n])
            y_pred = output.data.max(1, keepdim=True)[1]
            correct += y_pred.eq(y_test[n]).sum().item()
    print('Accuracy:', correct / len(y_test))
    print(time.time() - byrjun)
    print('----')

Epoch:  0
Accuracy: 0.421875
539.2964789867401
----
Epoch:  1
Accuracy: 0.375
1067.8597650527954
----
Epoch:  2
Accuracy: 0.375
1586.2667210102081
----
Epoch:  3
Accuracy: 0.375
2109.4060530662537
----
Epoch:  4
Accuracy: 0.375
2664.4733459949493
----
Epoch:  5
Accuracy: 0.375
3222.6761951446533
----
Epoch:  6
Accuracy: 0.375
3784.357887983322
----
Epoch:  7
Accuracy: 0.375
4324.138797044754
----
Epoch:  8
Accuracy: 0.375
4862.470696926117
----
Epoch:  9
Accuracy: 0.375
5372.754050016403
----


## Lokaorð

Líkanið spáir rétta flokkinn fyrir 42% myndanna eftir 1 lotu ("epoch"). En ef það væri af handahófi, þá myndi það vera 20%. Líkanið virkar.

Hvernig notum við líkan eins og þetta?
- Til að greina þrívíddarmyndir sem notendur teikna.
- Sem fyrsta skref í stærra líkani sem teiknir sjálft eftir texta sem notendur skrifa.

Vandamál:
- Ekki nóg gögn. Alvörulíkön nota fleiri en 10.000 mynda til að þjálfast.
- Ekki nógir flokkar. Allir flokkar, sérstaklega "líkamshlutar", eru mjög fjölbreyttir og alls ekki eins fyrir tölvuna.
- Við notum aðeins punkta, ekki línur eða flatir, til að greina.

En þetta er það besta fyrir gögnin sem ég var með. Og það tók nóg langan tíma til að þjálfa líkanið; þess vegna þurfum við betri tölvu.